# Import Libraries

In [1]:
import pandas as pd
import numpy as np

import glob
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Features to be used

**Fatigue**
- Days since last match
- Distance covered in the last x matches?

**Home Team Form**
- Goals difference of home team in the last x matches    
- Goals difference of home team in the last x home matches    
- Average number of points gained by home team in the last x matches
- Number of home matches won by home team in its last x home matches
- Home Team Win streak  
- Home Team Newly Promoted Team?

**Away Team Form**
- Goals difference of away team in the last x matches  
- Goals difference of away team in the last x away matches
- Average number of points gained by away team in the last x matches
- Number of away matches won by away team in its last x away matches
- Away Team Win streak
- Away Team Newly Promoted Team?

**Home Team Performance Index**
- Home Defense Performance Index
- Home Midfield Performance Index
- Home Attack Performance Index

**Away Team Performance Index**
- Away Defense Performance Index
- Away Midfield Performance Index
- Away Attack Performance Index

**Betting Odds**
- B365H
- B365D
- B365A

# Data Preparation

There are two main data sources for this project: 
- www.football-data.co.uk 
- www.fbref.com/en. 

Both of these two data sources provide match statistics data for Premier League matches. We will extract match statistics from both of these websites and integrate them into one DataFrame.

First we create an empty DataFrame that will eventually contain the data integrated from the various data sources.

In [3]:
df = pd.DataFrame()

## Data Source 1

www.football-data-co.uk

In [4]:
datasource1_df = pd.DataFrame()
seasons = ['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

for season in seasons:
    # read csv file for match statistics
    temp_df = pd.read_csv(f'datasets/{season}/data-source-1/{season}-match-statistics.csv')

    # drop irrelvant columns
    temp_df.drop(temp_df.loc[:, 'BWH':].columns, axis=1, inplace = True)
    temp_df.drop(['Div', 'HTHG', 'HTAG', 'HTR', 'Referee', 'Time'], axis=1, inplace = True, errors='ignore')
    
    # convert 'Date' column to datetime object
    temp_df['Date'] =  pd.to_datetime(temp_df['Date'], format="%d/%m/%Y")
    
    # concatenate temp_df to datasource1_df
    if datasource1_df.empty:
        datasource1_df = temp_df
    else:
        datasource1_df = pd.concat([datasource1_df, temp_df]).reset_index(drop=True)

In [5]:
# Make sure we have 5 x 380 = 1900 matches in the DataFrame
datasource1_df.shape

(1900, 21)

In [6]:
# standardize the teams names across all datasets

rename_teams = {'Arsenal': 'arsenal', 'Brighton': 'brighton', 'Chelsea': 'chelsea', 'Crystal Palace': 'palace', 'Everton': 'everton', 
                'Southampton': 'southampton', 'Watford': 'watford', 'West Brom': 'west-brom', 'Man United': 'united', 'Newcastle': 'newcastle',
                'Bournemouth': 'bournemouth', 'Burnley': 'burnley', 'Leicester': 'leicester', 'Liverpool': 'liverpool', 'Stoke': 'stoke',
                'Swansea': 'swansea', 'Huddersfield': 'huddersfield', 'Tottenham': 'tottenham', 'Man City': 'city', 'West Ham': 'west-ham',
                'Fulham': 'fulham', 'Wolves': 'wolves', 'Cardiff': 'cardiff', 'Aston Villa': 'aston-villa', 'Norwich': 'norwich',
                'Sheffield United': 'sheffield', 'Leeds': 'leeds', 'Brentford':'brentford'}

datasource1_df['HomeTeam'] = datasource1_df['HomeTeam'].apply(lambda word : rename_teams[word])
datasource1_df['AwayTeam'] = datasource1_df['AwayTeam'].apply(lambda word : rename_teams[word])

In [7]:
datasource1_df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,2017-08-11,arsenal,leicester,4,3,H,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.5,6.50
1,2017-08-12,brighton,city,0,2,A,6,14,2,4,6,9,3,10,0,2,0,0,11.00,5.5,1.33
2,2017-08-12,chelsea,burnley,2,3,A,19,10,6,5,16,11,8,5,3,3,2,0,1.25,6.5,15.00
3,2017-08-12,palace,huddersfield,0,3,A,14,8,4,6,7,19,12,9,1,3,0,0,1.83,3.6,5.00
4,2017-08-12,everton,stoke,1,0,H,9,9,4,1,13,10,6,7,1,1,0,0,1.70,3.8,5.75


## Data Source 2
www.fbref.com/en

In [8]:
datasource2_df = pd.DataFrame()
seasons = ['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

for season in seasons:

    # get all the datasets from data-source-2 folder
    path = f'datasets/{season}/data-source-2'
    csv_files = glob.glob(os.path.join(path, "*.csv"))

    for file in csv_files:
        team = file.split('\\')[1].split('-fixtures')[0]
        
        # read csv file for each team
        temp_df = pd.read_csv(file)

        # drop irrelvant columns
        temp_df.drop(['Time', 'Day', 'Attendance', 'Captain', 'Formation', 'Referee', 'Match Report', 'Notes', 'Round'], axis=1, inplace = True)

        # filter by Premier League home matches only
        temp_df = temp_df[(temp_df['Comp'] == 'Premier League') & (temp_df['Venue'] == 'Home')]

        # rename features
        temp_df = temp_df.rename(columns={'xG': 'HxG', 'xGA': 'AxG', 'Poss': 'HPoss', 'Opponent': 'AwayTeam'})

        # add new feature: 'HomeTeam'
        temp_df['HomeTeam'] = team

        # drop home team column
        temp_df.drop(['Venue'], axis=1, inplace = True)

        # add new feature: 'APoss'
        temp_df['APoss'] = 100 - temp_df['HPoss']

        # concatenate temp_df to datasource2_df
        if datasource2_df.empty:
            datasource2_df = temp_df
        else:
            datasource2_df = pd.concat([datasource2_df, temp_df]).reset_index(drop=True)

In [9]:
# Make sure we have 5 x 380 = 1900 matches in the DataFrame
datasource2_df.shape

(1900, 12)

In [10]:
datasource2_df['AwayTeam'].unique()

array(['Leicester City', 'Bournemouth', 'West Brom', 'Brighton',
       'Swansea City', 'Tottenham', 'Huddersfield', 'Manchester Utd',
       'Newcastle Utd', 'Liverpool', 'Chelsea', 'Crystal Palace',
       'Everton', 'Manchester City', 'Watford', 'Stoke City',
       'Southampton', 'West Ham', 'Burnley', 'Arsenal', 'Wolves',
       'Fulham', 'Cardiff City', 'Aston Villa', 'Sheffield Utd',
       'Norwich City', 'Leeds United', 'Brentford'], dtype=object)

In [11]:
# standardize the teams names across all datasets

rename_teams = {'Leicester City':'leicester', 'Bournemouth':'bournemouth', 'West Brom':'west-brom', 'Brighton': 'brighton', 'Swansea City':'swansea', 
                'Tottenham':'tottenham', 'Huddersfield':'huddersfield', 'Manchester Utd': 'united', 'Newcastle Utd':'newcastle', 'Liverpool':'liverpool', 
                'Chelsea': 'chelsea', 'Crystal Palace': 'palace', 'Everton': 'everton', 'Manchester City':'city', 'Watford': 'watford', 
                'Stoke City':'stoke', 'Southampton': 'southampton', 'West Ham':'west-ham', 'Burnley':'burnley', 'Arsenal': 'arsenal', 
                'Wolves':'wolves', 'Fulham':'fulham', 'Cardiff City':'cardiff', 'Aston Villa':'aston-villa', 'Sheffield Utd':'sheffield',
                'Norwich City':'norwich', 'Leeds United':'leeds', 'Brentford':'brentford'}

datasource2_df['AwayTeam'] = datasource2_df['AwayTeam'].apply(lambda word : rename_teams[word])

In [12]:
datasource2_df.head()

,Date,Comp,Result,GF,GA,AwayTeam,HxG,AxG,HPoss,DaysLastPlayed,HomeTeam,APoss
0,2017-08-11,Premier League,W,4,3,leicester,2.5,1.5,68.0,5 days,arsenal,32.0
1,2017-09-09,Premier League,W,3,0,bournemouth,2.2,0.6,58.0,13 days,arsenal,42.0
2,2017-09-25,Premier League,W,2,0,west-brom,2.2,0.9,69.0,5 days,arsenal,31.0
3,2017-10-01,Premier League,W,2,0,brighton,2.4,0.4,64.0,3 days,arsenal,36.0
4,2017-10-28,Premier League,W,2,1,swansea,2.0,0.9,72.0,4 days,arsenal,28.0


# Integration of Data Sources 

In [20]:
# df

In [21]:
# df = datasource1_df
# df.head()

In [22]:
# df = pd.merge(df, datasource2_df, on=['Date', 'HomeTeam', 'AwayTeam'])

In [23]:
# df.head()

# Feature Engineering

In [ ]:
# add new feature: 'DaysLastPlayed'
# temp_df['Date'] =  pd.to_datetime(temp_df['Date'], format="%Y/%m/%d")
# temp_df['HT_DaysLastPlayed'] = temp_df['Date'] - temp_df['Date'].shift(1)